In [2]:
import time
import os
import math
import tushare as ts
from datetime import datetime
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
from IPython.display import clear_output

sys.path.append("../utils/")
import date_util
import token_util
import enter_util
import stock_util

### 提取数据

In [3]:
trade_date = "20190830"
delta_price = 30.0

In [ ]:
# stock_list = stock_util.get_stock_list(trade_date, delta_price)

In [4]:
stock_list = pd.read_csv("../data_pulled/stock_{}.csv".format(trade_date))
stock_list.head()

,ts_code,name,price
0,000001.SZ,平安银行,14.145
1,000002.SZ,万科A,25.885
2,000004.SZ,国农科技,20.090
3,000005.SZ,世纪星源,3.135
4,000006.SZ,深振业A,5.360


### 选出符合入场点的股票

In [60]:
i = 2
trade_date = '20011220'
stock_code = stock_list.iloc[i]["ts_code"]
    
if stock_code + ".csv" == stock_csv_list[i+1]:
    cur_detail = pd.read_csv(os.path.join(ROOT_PATH, stock_csv_list[i+1]))
    cur_index = str(cur_detail["trade_date"]) == trade_date

In [61]:
cur_detail[str(cur_detail["trade_date"]) == trade_date]

KeyError: False

In [51]:
str(cur_detail["trade_date"][0])

'20011220'

In [32]:
cur_detail[cur_index]

,Unnamed: 0,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,diff,dea,macd,ema7,ema14


In [4]:
# 选股

trad_date = '20190930'

golden_list = []

columns = ["stock_code", "pct_chg_short", "his_macd_discount", "amount_mean"]
choose_list = pd.DataFrame(columns = columns)

ROOT_PATH = "../data_pulled/day"
stock_csv_list = os.listdir(ROOT_PATH)


for i in range(len(stock_list)):
    
    stock_code = stock_list.iloc[i]["ts_code"]
    
    if stock_code + ".csv" == stock_csv_list[i+1]:
        cur_detail = pd.read_csv(os.path.join(ROOT_PATH, stock_csv_list[i+1]))
        cur_index = cur_detail["trade_date"] == trad_date

    if ma7_1 < ma14_1 and ma7_2 > ma14_2 :

        golden_list.append(stock_code)
        macd, data_macd, his_macd_discount, macd_trend = enter_util.cal_macd(stock_code, end_date, multi_output=True)
        list_tmp = pd.DataFrame([[stock_code, ptc_chg_short, his_macd_discount, amount_mean]], columns=columns)

        if macd > 0:
            if macd_trend:
                choose_list = choose_list.append(list_tmp)

    
    if i % 100 == 0:
        clear_output(True)
        print(i)
        print(choose_list)

# 数据归一化

for column in choose_list.columns[1:]:
    c_data = choose_list[column]
    choose_list[column] = (c_data-c_data.min())/(c_data.max() - c_data.min())

# 排序

def get_rank_factor(arr):
    pct_chg_short = arr["pct_chg_short"] * 0.5
    his_macd_discount = arr["his_macd_discount"]
    amount_mean = arr["amount_mean"]
    return pct_chg_short+his_macd_discount+amount_mean

choose_list["rank_factor"] = choose_list.apply(get_rank_factor, axis = 1)
choose_list = choose_list.sort_values(by="rank_factor", ascending= False).reset_index(drop=True)

2600
  stock_code  pct_chg_short  his_macd_discount    amount_mean
0  000422.SZ         1.7643           0.557250   20687.255818
0  000429.SZ         0.3923           0.541750   25152.578727
0  000611.SZ         3.5079           0.544000    8469.332364
0  000707.SZ         6.4033           0.539250   17007.624273
0  000806.SZ         3.0364           0.561750   26268.826000
0  000895.SZ         2.9194           0.517750  371628.446818
0  002111.SZ        -0.5079           0.532030  227446.760818
0  002324.SZ         1.9616           0.554681   55106.132636
0  002351.SZ         7.7627           0.552052   99601.748000
0  002563.SZ         4.6279           0.525399   75795.938545
0  002627.SZ         3.9481           0.543592   15941.752909
0  002739.SZ         6.5093           0.565714  313876.144909
0  002741.SZ         2.2792           0.537396   93813.375364
0  002873.SZ         7.9619           0.584483   30644.369727
0  600201.SH         3.1303           0.530750  377268.151455
0  

# TEST